In [96]:
def viterbi(initial_conditions, likelihood, transition_matrix):

    LOG_EPS = 1e-16
    n_time, n_states = likelihood.shape

    log_likelihood = np.log(likelihood + LOG_EPS)
    log_state_transition = np.log(transition_matrix + LOG_EPS)
    log_initial_conditions = np.log(initial_conditions + LOG_EPS)

    path_log_prob = np.ones_like(likelihood)
    back_pointer = np.zeros_like(likelihood, dtype=int)

    path_log_prob[0] = log_initial_conditions + log_likelihood[0]

    for time_ind in range(1, n_time):
        prior = path_log_prob[time_ind - 1] + log_state_transition
        for state_ind in range(n_states):
            back_pointer[time_ind, state_ind] = np.argmax(prior[state_ind])
            path_log_prob[time_ind, state_ind] = (
                prior[state_ind, back_pointer[time_ind, state_ind]]
                + log_likelihood[time_ind, state_ind]
            )

    # Find the best accumulated path prob in the last time bin
    # and then trace back the best path
    best_path = np.zeros((n_time,), dtype=int)
    best_path[-1] = np.argmax(path_log_prob[-1])
    for time_ind in range(n_time - 2, -1, -1):
        best_path[time_ind] = back_pointer[time_ind + 1, best_path[time_ind + 1]]

    return best_path, np.exp(np.max(path_log_prob[-1]))

In [88]:
from src.test_hmm import generate_data, generate_data3

(
    initial_conditions,
    transition_matrix,
    emission_matrix,
    _,
    observations_ind,
) = generate_data3()

likelihood = get_likelihood(emission_matrix, observations_ind)

best_path, path_prob = viterbi(initial_conditions, likelihood, transition_matrix)
best_path, path_prob

(array([0, 2, 2, 1, 1, 1, 1, 1, 0]), 4.3584805012498837e-20)

In [101]:
from src.hmm import get_likelihood
from src.test_hmm import generate_data, generate_data3

# prob_pathefine model parameters
transition_matrix = np.array([[0.8, 0.1, 0.1], 
              [0.2, 0.7, 0.1], 
              [0.1, 0.3, 0.6]])

initial_conditions = np.array([0.6, 0.2, 0.2])

emission_matrix = np.array([[0.7, 0.0, 0.3], 
                            [0.1, 0.9, 0.0], 
                            [0.0, 0.2, 0.8]])


observations_ind = np.array([0, 2, 0, 2, 2, 1]).astype(np.int32)

likelihood = get_likelihood(emission_matrix.T, observations_ind)

best_path, path_prob = viterbi(initial_conditions, likelihood, transition_matrix)
best_path, path_prob

# [0.336 0.004 0.   ]
# [0.042 0.014 0.   ]
# [0.042 0.002 0.   ]

(array([0, 2, 2, 2, 2, 1]), 0.00012541132800000045)

In [1]:
from src.hmm import viterbi
import numpy as np
from src.hmm import get_likelihood

# Define model parameters
transition_matrix = np.array([[0.8, 0.1, 0.1], 
              [0.2, 0.7, 0.1], 
              [0.1, 0.3, 0.6]])

initial_conditions = np.array([0.6, 0.2, 0.2])

emission_matrix = np.array([[0.7, 0.0, 0.3], 
              [0.1, 0.9, 0.0], 
              [0.0, 0.2, 0.8]])


observations_ind = np.array([0, 2, 0, 2, 2, 1]).astype(np.int32)

likelihood = get_likelihood(emission_matrix, observations_ind)

viterbi(initial_conditions, likelihood, transition_matrix)

(array([0, 0, 0, 0, 0, 0]), 2.6011238399999947e-19)

In [3]:
def viterbi2(initial_conditions, likelihood, transition_matrix):

    LOG_EPS = 1e-16
    n_time, n_states = likelihood.shape

    log_likelihood = np.log(likelihood + LOG_EPS)
    log_state_transition = np.log(transition_matrix + LOG_EPS)
    log_initial_conditions = np.log(initial_conditions + LOG_EPS)

    path_log_prob = np.ones_like(likelihood)
    back_pointer = np.zeros_like(likelihood, dtype=int)

    path_log_prob[0] = log_initial_conditions + log_likelihood[0]

    for time_ind in range(1, n_time):
        prior = path_log_prob[time_ind - 1] + log_state_transition
        for state_ind in range(n_states):
            back_pointer[time_ind, state_ind] = np.argmax(prior[state_ind])
            path_log_prob[time_ind, state_ind] = (
                prior[state_ind, back_pointer[time_ind, state_ind]]
                + log_likelihood[time_ind, state_ind]
            )

    # Find the best accumulated path prob in the last time bin
    # and then trace back the best path
    best_path = np.empty((n_time,), dtype=int)
    best_path[-1] = np.argmax(path_log_prob[-1])
    for time_ind in range(n_time - 2, -1, -1):
        best_path[time_ind] = back_pointer[time_ind + 1, best_path[time_ind + 1]]

    return best_path, np.exp(np.max(path_log_prob[-1]))

In [4]:
viterbi2(initial_conditions, likelihood, transition_matrix)

(array([0, 0, 0, 0, 0, 1]), 0.0005852528640000015)

In [ ]:
def hmm_information_criterion(log_likelihood, n_states, n_independent_parameters, n_time=1):
    n_parameters = n_states**2 + n_independent_parameters * n_states - 1
    aic = -2 * log_likelihood + 2 * n_parameters
    bic = -2 * log_likelihood + n_parameters * np.log(n_time)
    
    return aic, bic